# 投资组合优选优化

本模型是经典的[马科维茨投资组合选择优化模型](https://en.wikipedia.org/wiki/Markowitz_model)的示例。我们要在一组股票中找到投资组合的比例，以平衡风险和回报。这是一个具有向量和矩阵数据（分别用于回报和风险）的二次规划（QP）模型。这最适合用矩阵形式表示，因此我们使用 Gurobi Python 的*矩阵*接口。基本模型相当简单，所以我们还通过参数化求解来找到有效前沿。

**下载代码库** <br /> 
你可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 模型构建
### 参数

我们使用金融领域传统的[希腊字母值](https://en.wikipedia.org/wiki/Greeks_\(finance\))：

- $\delta$: 测量每支股票价格变化的 n 维向量
- $\sigma$: 测量股票之间协方差的 n x n 矩阵

在参数化求解模型时还有一个额外的参数：

- r: 目标收益率

### 决策变量
- $x \ge 0$: n维向量，每个元素代表投资组合中投资于每支股票的比例

### 目标函数
最小化总风险，这是一个凸二次函数：

\begin{equation}
\min x^t \cdot \sigma \cdot x
\end{equation}

### 约束条件

分配整个投资组合：总投资应为1.0（100%），其中 $e$ 是单位向量（全为1）：

\begin{equation}
e \cdot x = 1
\end{equation}

收益率：当我们对不同的收益率值 $r$ 进行参数化求解时，我们添加目标收益率的约束：

\begin{equation}
\delta \cdot x = r
\end{equation}

## Python 实现
### 股票数据
使用 [yfinance](https://pypi.org/project/yfinance/) 库获取美国20家最赚钱公司（[根据2021年4月的维基百科](https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue#List_of_companies_by_profit)）最近2年的*实际股票数据*。

In [ ]:
%pip install gurobipy yfinance

In [ ]:
import yfinance as yf

stocks = ['BRK-A', 'AAPL', 'MSFT', 'JPM', 'GOOG', 'BAC', 'INTC', 'WFC',
          'C', 'VZ', 'META', 'PFE', 'JNJ', 'WMT', 'XOM',
          'FNMA', 'T', 'UNH', 'CMCSA', 'V' ]

data = yf.download(stocks, period='2y')

### 计算希腊字母值
使用下载的股票数据，计算股票价格的 delta（收益率）、sigma（协方差）和标准差值：

In [ ]:
import numpy as np

closes = np.transpose(np.array(data.Close)) # 每日收盘价矩阵
absdiff = np.diff(closes)                   # 每天收盘价的变化
reldiff = np.divide(absdiff, closes[:,:-1]) # 每日收盘价的相对变化
delta = np.mean(reldiff, axis=1)            # 平均价格变动
sigma = np.cov(reldiff)                     # 协方差（标准差）
std = np.std(reldiff, axis=1)               # 标准偏差

## 通过求解 QP 模型最小化风险

In [ ]:
import gurobipy as gp
from gurobipy import GRB
from math import sqrt

# 创建空模型
m = gp.Model('portfolio')

# 为股票添加矩阵变量
x = m.addMVar(len(stocks))

# 目标是最小化风险（平方）。这是使用协方差矩阵建模的，
# 协方差矩阵用于衡量股票之间的历史相关性
portfolio_risk = x @ sigma @ x
m.setObjective(portfolio_risk, GRB.MINIMIZE)

# 用约束条件固定预算
m.addConstr(x.sum() == 1, 'budget')

# 验证模型公式
m.write('portfolio_selection_optimization.lp')

# 优化模型以找到最小风险投资组合
m.optimize()

## 使用 Pandas 显示最小风险投资组合

In [ ]:
import pandas as pd
minrisk_volatility = sqrt(m.ObjVal)
minrisk_return = delta @ x.X
pd.DataFrame(data=np.append(x.X, [minrisk_volatility, minrisk_return]),
             index=stocks + ['Volatility', 'Expected Return'],
             columns=['Minimum Risk Portfolio'])

## 计算有效前沿
通过参数化求解 QP 模型，找到不同预期收益率下的最低风险投资组合。

In [ ]:
# 创建表示投资组合预期回报的表达式
portfolio_return = delta @ x
target = m.addConstr(portfolio_return == minrisk_return, 'target')

# 通过改变目标回报求解有效前沿
frontier = np.empty((2,0))
for r in np.linspace(delta.min(), delta.max(), 25):
    target.rhs = r
    m.optimize()
    frontier = np.append(frontier, [[sqrt(m.ObjVal)],[r]], axis=1)

## 绘制结果
使用 matplot 库绘制优化解和个股的图表：

In [ ]:
import matplotlib.pyplot as plt
#plt.figure(figsize=(10,10))

fig, ax = plt.subplots(figsize=(10,8))

# 绘制个股的波动率与预期回报关系图
ax.scatter(x=std, y=delta,
           color='Blue', label='Individual Stocks')
for i, stock in enumerate(stocks):
    ax.annotate(stock, (std[i], delta[i]))

# 绘制波动性与最小风险投资组合的预期回报
ax.scatter(x=minrisk_volatility, y=minrisk_return, color='DarkGreen')
ax.annotate('Minimum\nRisk\nPortfolio', (minrisk_volatility, minrisk_return),
            horizontalalignment='right')

# 绘制有效前沿
ax.plot(frontier[0], frontier[1], label='Efficient Frontier', color='DarkGreen')

# 设置和显示最终图表格式
ax.axis([frontier[0].min()*0.7, frontier[0].max()*1.3, delta.min()*1.2, delta.max()*1.2])
ax.set_xlabel('Volatility (standard deviation)')
ax.set_ylabel('Expected Return')
ax.legend()
ax.grid()
plt.show()